In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Neccesary Imports
import os
import cv2
import wandb
import time
import torch
import torch.nn as nn
import numpy as np
import torchvision
import matplotlib
import matplotlib.pyplot as plt
import albumentations as A
import torch.optim as optim
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import torch.nn.functional as F
from tqdm.auto import tqdm

In [ ]:
import torch
torch.manual_seed(42)
#42? You know why 

In [ ]:
original_height=384
original_width =384

#Albumentations Transformations
transformations = A.Compose([
    A.VerticalFlip(p=0.5),              
    A.RandomRotate90(p=0.5),
    A.RandomGamma(p=0.8),
    ])
#Highly Inspired from the Original shared notebook : 38-Cloud-Data preparation by cordmau
class Clouds(Dataset):
    def __init__(self, r_dir, g_dir, b_dir, nir_dir, gt_dir,transform=False):
        super().__init__()
        
        # Looping through the files in red folder and combine, into a dictionary, the other bands
        self.files = [self.combine_files(f, g_dir, b_dir, nir_dir, gt_dir) for f in r_dir.iterdir() if not f.is_dir()]
        self.transform = transform
        
    def combine_files(self, r_file: Path, g_dir, b_dir,nir_dir, gt_dir):
        
        files = {'red': r_file, 
                 'green':g_dir/r_file.name.replace('red', 'green'),
                 'blue': b_dir/r_file.name.replace('red', 'blue'), 
                 'nir': nir_dir/r_file.name.replace('red', 'nir'),
                 'gt': gt_dir/r_file.name.replace('red', 'gt')}

        return files
                                       
    def __len__(self):
        
        return len(self.files)
     
    def get_image(self, idx):

        image = np.stack([np.array(Image.open(self.files[idx]['red'])),
                            np.array(Image.open(self.files[idx]['green'])),
                            np.array(Image.open(self.files[idx]['blue'])),
                           ], axis=2)
        
        nir = np.expand_dims(np.array(Image.open(self.files[idx]['nir'])), 2)
        image = np.concatenate([nir,image], axis=2)
            
        # normalizing according to the paper
        return (image / np.iinfo(image.dtype).max)
    

    def get_mask(self, idx):
        
        mask = np.array(Image.open(self.files[idx]['gt']))
        mask = np.where(mask==255, 1, 0)#Semantic Segmentaion, so 1- Clouds, 0-Background.
        return mask
    
    def __getitem__(self, idx):
                    
        x = self.get_image(idx)
        
        y = self.get_mask(idx)
       
        if self.transform is not None:
            augmented = self.transform(image=x,mask=y)
            x=augmented['image']
            y=augmented['mask']
            
        x=torch.from_numpy(x)
        y=torch.tensor(y,dtype=torch.float32) 
        x = x.permute(2,0,1)#Numpy -> PyTorch
        
        return x.double(), y.unsqueeze(0)#Need the mask to be of [1,H,W] beacuse of output of the network is of similar dimensions
    
    def open_as_pil(self, idx):
        
        arr = 256*self.open_as_array(idx)
        
        return Image.fromarray(arr.astype(np.uint8), 'RGB')
    
    def __repr__(self):
        s = 'Dataset class with {} files'.format(self.__len__())

        return s
#MY OWN VERSION BUT IT IS TO A BIT SLOW
original_height=384
original_width =384
original_height=256
original_width =256

transformed = A.Compose([
    A.Resize(256,256),
    A.OneOf([
        A.RandomSizedCrop(min_max_height=(50, 101), height=original_height, width=original_width, p=0.5),
        A.PadIfNeeded(min_height=original_height, min_width=original_width, p=0.5)
    ], p=1),    
    A.VerticalFlip(p=0.5),              
    A.RandomRotate90(p=0.5),
    A.OneOf([
        A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.5),
        A.GridDistortion(p=0.5),
        A.OpticalDistortion(distort_limit=2, shift_limit=0.5, p=1)                  
        ], p=0.8),
    #A.CLAHE(p=0.8),
    #A.RandomBrightnessContrast(p=0.8),    
    A.RandomGamma(p=0.8),
    ])

def get_channels(path):
    red_ch=os.listdir(path+'train_red')
    blue_ch=os.listdir(path+'train_blue')
    green_ch=os.listdir(path+'train_green')
    nir_ch=os.listdir(path+'train_nir')
    masks=os.listdir(path+'train_gt')
    return red_ch,blue_ch,green_ch,nir_ch,masks

#f=transforms.ToTensor()
def f(image):
    return np.array(image)

def get_image(path,r,b,g,n,m):
    red=f(Image.open(path+'train_red/'+r))
    blue=f(Image.open(path+'train_blue/'+b))
    green=f(Image.open(path+'train_green/'+g))
    nir=f(Image.open(path+'train_nir/'+n))
    mask=f(Image.open(path+'train_gt/'+m))
    return red,blue,green,nir,mask

path='../input/38cloud-cloud-segmentation-in-satellite-images/38-Cloud_training/'

class Clouds(Dataset):
    
    def __init__(self,path,transform=None,num_channels=4):
        
        self.path=path
        self.transform=transform
        self.channels=get_channels(path)
        self.num_channels=num_channels
       
    def process(self,r,g,b,n,m):
        r,g,b,n,m=get_image(path,r,g,b,n,m)
        image = np.stack([n,r,g],axis=2)
        mask=m
        return image,mask
        
    def __getitem__(self,idx):
        
        r=self.channels[0][idx]
        g=self.channels[1][idx]
        b=self.channels[2][idx]
        n=self.channels[3][idx]
        m=self.channels[4][idx]
        
        image,mask=self.process(r,g,b,n,m)
        image = image.astype('uint16')
        image = image / np.iinfo(image.dtype).max
        mask  = mask/255.0
       
        if self.transform is not None:
      
            augmented = self.transform(image=image,mask=mask)
            image=torch.from_numpy(augmented['image']).permute(2,0,1)
            mask=torch.from_numpy(augmented['mask']) 
            
        image=image.double()
        mask=mask.unsqueeze(0)
        mask = mask.type(torch.float32)
        return image,mask
    
    def __len__(self):
        return len(self.channels[0])
       

#dataset=Clouds(path,transform=transformed)
#image,mask=dataset[2]
#plt.imshow(image.permute(1,2,0))

In [ ]:
image,label=dataset[2]
plt.imshow(image.permute(1,2,0))

In [ ]:
base_path = Path('../input/38cloud-cloud-segmentation-in-satellite-images/38-Cloud_training')
dataset = CloudDataset(base_path/'train_red', 
                    base_path/'train_green', 
                    base_path/'train_blue', 
                    base_path/'train_nir',
                    base_path/'train_gt',transform=transformations)


In [ ]:
train_length=int(0.712* len(dataset))

test_length=len(dataset)-train_length


train_dataset,test_dataset=torch.utils.data.random_split(dataset,(train_length,test_lengt))
batch_size= 12 

trainloader = DataLoader(train_dataset,
        batch_size=batch_size, shuffle=True,num_workers= 2)
testloader = DataLoader(test_dataset,
        batch_size=batch_size, shuffle=False,num_workers=2)

In [ ]:
def Metrics(inputs, targets):
    
    inputs = torch.sigmoid(inputs)
    inputs=inputs.round().int()
    targets=targets.int()
    smooth = 1.0
    inputs = inputs.view(-1)
    targets = targets.view(-1)
    intersection = (inputs & targets).float().sum()
    TP=intersection
    FP = ((1-targets) & inputs).float().sum()
    FN = (targets & (1-inputs)).float().sum()
    TN = ((1-targets) & (1-inputs)).float().sum()
    total = (inputs + targets).float().sum()
    union = total - intersection 
    IoU = ((intersection + smooth)/(union + smooth))
    # 'Dice':dice, 'Pixel_Acc': acc, 'Precision': precision,'Recall': recall, 'Specificity':specificity}
    #return metrics
    
    dice = (2.0 * intersection + smooth) / (total + smooth)
    IoU = ((intersection + smooth)/(union + smooth))
    valid = (targets >= 0)
    acc_sum = (valid * (inputs == targets)).sum()
    valid_sum = valid.sum()
    #acc2=(inputs.argmax(dim=1) == targets.float().mean()
    acc = (float(acc_sum) / (valid_sum + 1e-10))
    precision = (intersection/(FP+intersection+1e-5))
    recall = (intersection/(FN+intersection+1e-5))
    specificity = (TN/(TN+FP+1e-5))
    metrics={'IoU':IoU,'Dice':dice, 'Pixel_Acc': acc, 'Precision': precision,'Recall': recall, 'Specificity':specificity}
    return metrics


In [ ]:
#Sorry for this, I have attached the scratch implementations which failed in the bottom.
!pip install segmentation_models_pytorch 
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7,     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=4,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)

#mask=torch.randn(4,1,384,384)
#target=torch.randn(4,1,384,384)


#z=m(target,mask)
device = "cpu"

if torch.cuda.is_available():
    device = "cuda"

print(device)

In [ ]:
def Metrics(inputs, targets):
    inputs = torch.sigmoid(inputs)
    inputs=inputs.round().int()
    targets=targets.round().int()
    smooth = 1.0
    inputs = inputs.view(-1)
    targets = targets.view(-1)
    intersection = (inputs & targets).float().sum()
    TP=intersection
    FP = ((1-targets) & inputs).float().sum()
    FN = (targets & (1-inputs)).float().sum()
    TN = ((1-targets) & (1-inputs)).float().sum()
    total = (inputs + targets).float().sum()
    union = total - intersection 
    dice = (2.0 * intersection + smooth) / (total + smooth)
    IoU = ((intersection + smooth)/(union + smooth))
    valid = (targets >= 0)
    acc_sum = (valid * (inputs == targets)).sum()
    valid_sum = valid.sum()
    acc = (float(acc_sum) / (valid_sum + 1e-10))
    precision = (intersection/(FP+intersection+1e-5))
    recall = (intersection/(FN+intersection+1e-5))
    specificity = (TN/(TN+FP+1e-5))
    metrics={'IoU':IoU, 'Dice':dice, 'Pixel_Acc': acc, 'Precision': precision,'Recall': recall, 'Specificity':specificity}
    return metrics


In [ ]:

def train(network,criterion, optimizer, trainloader):
    loss_train = 0
    acc_train = 0
    network.train()
    
    for step in tqdm(range(len(trainloader))):

        images , masks = next(iter(trainloader))
        
        # move the images and labels to GPU
        images = images.to(device)
        masks = masks.to(device)
      

        pred = network(images)
        #pred=torch.sigmoid(pred)
      
        # clear all the gradients before calculating them
        optimizer.zero_grad()
        #print(masks.shape,images.shape)
        # find the loss for the current step
        loss_train_step= criterion(pred , masks)
        
        # find accuracy
        acc_train_ = Metrics(pred,masks)
        acc_train_step=acc_train_['IoU']
        # calculate the gradients
        loss_train_step.backward()
        #print(loss_train_step,masks.shape)
        #print(acc_train_)
        # update the parameters
        optimizer.step()
        
        loss_train += loss_train_step.item()
        acc_train += acc_train_step  
            
    loss_train /= len(trainloader)
    acc_train /= len(trainloader)

    return loss_train, acc_train,acc_train_  
        
def validate(network,criterion, testloader): 
    loss_valid = 0
    acc_valid = 0       
    network.eval()  

    for step in tqdm(range(len(testloader))):

        images , masks = next(iter(testloader))
        
        # move the images and labels to GPU
        images = images.to(device)
        masks = masks.to(device)
        
        
        pred = network(images)
        #pred=torch.sigmoid(pred)
        
      
        # clear all the gradients before calculating them
        optimizer.zero_grad()
        
        # find the loss and acc for the current step
        
        loss_valid_step = criterion(pred , masks)
        
        # find accuracy
        val_metrics=Metrics(pred,masks)
        acc_valid_step=val_metrics['IoU']
        acc_val = val_metrics
        loss_valid += loss_valid_step.item()
        acc_valid += acc_valid_step
        #print(loss_tvalid_step,masks.shape)
        #print(acc_val)

    loss_valid /= len(testloader)
    acc_valid /= len(testloader)

    return loss_valid, acc_valid,acc_val

In [ ]:
import wandb
wandb.login()

In [ ]:

wandb.init(name='Clouds', 
           project='UnNet+WeightedBCE',
           notes='RGBNIR', 
           #tags=['Replay-Attack','Cyclic_LR'],
           entity='creganstark')

# WandB Configurations (optional)        
wandb.config.lr = 5e-3 
#model=UNet()
model=model.double()
model = model.to(device)

#criterion = FocalTverskyLoss()
criterion=nn.BCEWithLogitsLoss(pos_weight=torch.tensor(2.3053))
optimizer = optim.Adam(model.parameters(),lr  =  wandb.config.lr)# ,
                      #momentum     = 0.9,
                      #nesterov     = True,
                      #weight_decay = 5e-4)

#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)


# Log the network weight histograms 
wandb.watch(model)

num_epochs = 20
start_time = time.time()
prev_acc=0
for epoch in range(1, num_epochs+1):
    
    loss_train, acc_train,metric_train = train(model, criterion, optimizer, trainloader)
    loss_valid, acc_valid,metric_val = validate(model, criterion, testloader)
    
    print('Epoch: {}  Train Loss: {:.4f}  Train IoU: {:.4f}  Valid Loss: {:.4f}  Valid IoU: {:.4f}'.
          format(epoch, loss_train, acc_train, loss_valid, acc_valid))

    # Log the loss and accuracy values at the end of each epoch
    wandb.log({
        "Epoch": epoch,
        "Train IoU": metric_train['IoU'],
        "Train Dice": metric_train['Dice'],
        "Train Pixel Acc": metric_train['Pixel_Acc'],
        "Train Precision":metric_train["Precision"],
        "Train Recall": metric_train['Recall'],
        "Train Specificity":metric_train['Specificity'],
        "Train Loss": loss_train,
        "Val IoU": metric_val['IoU'],
        "Val Dice ": metric_val['Dice'],
        "Val Pixel Acc": metric_val['Pixel_Acc'],
        "Val Precision":metric_val["Precision"],
        "Val Recall": metric_val['Recall'],
        "Val Specificity":metric_val['Specificity'],
        "Val Loss" : loss_valid
       })
    if acc_valid>prev_acc:
      prev_acc=acc_valid
      paths = "model"+str(acc_valid)+".pt"
      print('Saving Model')
      torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'val_loss': loss_valid,
          'val_acc':acc_valid,
          'train_acc':acc_train,
          'loss_acc':loss_train,
          }, str(paths))

print("Time Elapsed : {:.4f}s".format(time.time() - start_time))

Inference

In [ ]:
#For Loading from last checkpoint
filename='../input/finalbce50/finalbce50.pt'

def load_checkpoint(model,filepath):
    checkpoint = torch.load(filepath)
    model.load_state_dict(checkpoint['model_state_dict'])
    #for parameter in model.parameters():
        #parameter.requires_grad = True
    return model
#model=load_checkpoint(model,filename)

In [ ]:
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(2.3053))
model=model.double()
model = model.to(device)

In [ ]:
def Metrics(inputs, targets):
    
    inputs = torch.sigmoid(inputs)
    inputs=inputs.round().int()
    targets=targets.int()
    smooth = 1.0
    inputs = inputs.view(-1)
    targets = targets.view(-1)
    intersection = (inputs & targets).float().sum()
    TP=intersection
    FP = ((1-targets) & inputs).float().sum()
    FN = (targets & (1-inputs)).float().sum()
    TN = ((1-targets) & (1-inputs)).float().sum()
    total = (inputs + targets).float().sum()
    union = total - intersection 
    IoU = ((intersection + smooth)/(union + smooth))
    # 'Dice':dice, 'Pixel_Acc': acc, 'Precision': precision,'Recall': recall, 'Specificity':specificity}
    #return metrics
    
    dice = (2.0 * intersection + smooth) / (total + smooth)
    IoU = ((intersection + smooth)/(union + smooth))
    valid = (targets >= 0)
    acc_sum = (valid * (inputs == targets)).sum()
    valid_sum = valid.sum()
    #acc2=(inputs.argmax(dim=1) == targets.float().mean()
    acc = (float(acc_sum) / (valid_sum + 1e-10))
    precision = (intersection/(FP+intersection+1e-5))
    recall = (intersection/(FN+intersection+1e-5))
    specificity = (TN/(TN+FP+1e-5))
    metrics={'IoU':IoU,'Dice':dice, 'Pixel_Acc': acc, 'Precision': precision,'Recall': recall, 'Specificity':specificity}
    return metrics
def validate2(network,criterion, testloader): 
    
    #train_dataset,test_dataset=torch.utils.data.random_split(dataset,(6000,2400))
    

    #batch_size= 12

    
   # testloader = DataLoader(test_dataset,
        #batch_size=batch_size, shuffle=False,num_workers=2)
    
    
    loss_valid = 0
    acc_valid = 0    
    acc_pix = 0
    acc_dice=0
    precision=0
    recall=0
    specificity=0
    network.eval()  

    for step in tqdm(range(len(testloader))):

        images , masks = next(iter(testloader))
        
        # move the images and labels to GPU
        images = images.to(device)
        masks = masks.to(device)
        
        
        pred = network(images)
        
        
        loss_valid_step = criterion(pred , masks)
       
        val_metrics=Metrics(pred,masks)
        
        #acc_val = val_metrics
        acc_valid_step=val_metrics['IoU']
        acc_valid_dice=val_metrics['Dice']
        acc_valid_pixel=val_metrics['Pixel_Acc']
        acc_precision=val_metrics['Precision']
        acc_recall = val_metrics['Recall']
        acc_speicificity = val_metrics['Specificity']
        precision+=acc_precision
        recall+=acc_recall
        specificity+=acc_speicificity
        loss_valid += loss_valid_step.item()
        acc_valid += acc_valid_step
        acc_pix+=acc_valid_pixel
        acc_dice+=acc_valid_dice
        
        
        #print(loss_tvalid_step,masks.shape)
        #print(acc_val)

    loss_valid /= len(testloader)
    acc_valid /= len(testloader)
    acc_pix/= len(testloader)
    acc_dice/= len(testloader) 
    precision/=len(testloader) 
    recall/=len(testloader) 
    specificity/=len(testloader) 
    

    return loss_valid,acc_valid,acc_pix,acc_dice,precision,recall,specificity

In [ ]:
loss_valid,acc_valid,acc_pix,acc_dice,precision,recall,specificity=validate2(model,criterion, testloader)
loss_valid,acc_valid,acc_pix,acc_dice,precision,recall,specificity

Clutter from all the runs
Notebook ends here

In [ ]:
loss_valid,acc_valid,acc_pix,acc_dice,precision,recall,specificity

In [ ]:
loss_valid,acc_valid,acc_pix,acc_dice,precision,recall,specificity

In [ ]:
image,mask=dataset[2]
plt.imshow(image.permute(1,2,0))

In [ ]:
image,mask=dataset[150]
plt.imshow(image.permute(1,2,0))

In [ ]:
import wandb
wandb.login()


In [ ]:

def train(network,criterion, optimizer, trainloader):
    loss_train = 0
    acc_train = 0
    network.train()
    
    for step in tqdm(range(len(trainloader))):

        images , masks = next(iter(trainloader))
        
        # move the images and labels to GPU
        images = images.to(device)
        masks = masks.to(device)
      

        pred = network(images)
        #pred=torch.sigmoid(pred)
      
        # clear all the gradients before calculating them
        optimizer.zero_grad()
        #print(masks.shape,images.shape)
        # find the loss for the current step
        loss_train_step= criterion(pred , masks)
        
        # find accuracy
        acc_train_ = Metrics(pred,masks)
        acc_train_step=acc_train_['IoU']
        # calculate the gradients
        loss_train_step.backward()
        #print(loss_train_step,masks.shape)
        #print(acc_train_)
        # update the parameters
        optimizer.step()
        
        loss_train += loss_train_step.item()
        acc_train += acc_train_step  
            
    loss_train /= len(trainloader)
    acc_train /= len(trainloader)

    return loss_train, acc_train,acc_train_  
        
def validate(network,criterion, testloader): 
    loss_valid = 0
    acc_valid = 0       
    network.eval()  

    for step in tqdm(range(len(testloader))):

        images , masks = next(iter(testloader))
        
        # move the images and labels to GPU
        images = images.to(device)
        masks = masks.to(device)
        
        
        pred = network(images)
        #pred=torch.sigmoid(pred)
        
      
        # clear all the gradients before calculating them
        optimizer.zero_grad()
        
        # find the loss and acc for the current step
        
        loss_valid_step = criterion(pred , masks)
        
        # find accuracy
        val_metrics=Metrics(pred,masks)
        acc_valid_step=val_metrics['IoU']
     
        acc_val = val_metrics#Other metrics stored here
        loss_valid += loss_valid_step.item()
        acc_valid += acc_valid_step
        #print(loss_tvalid_step,masks.shape)
        #print(acc_val)

    loss_valid /= len(testloader)
    acc_valid /= len(testloader)

    return loss_valid, acc_valid,acc_val

In [ ]:
device = "cpu"

if torch.cuda.is_available():
    device = "cuda"
device

In [ ]:

import wandb

wandb.login()


In [ ]:
checkpoint = {'model': model,
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, 'epoch20checkpoint.pth')

In [ ]:
len(dataset)

In [ ]:
y=np.array([2,2,2])
y=torch.from_numpy(y)#,dtype=torch.torch.float32) 
y=y.to(torch.float32)

In [ ]:
y.dtype

In [ ]:
base_path = Path('../input/38cloud-cloud-segmentation-in-satellite-images/38-Cloud_training')
dataset = CloudDataset(base_path/'train_red', 
                    base_path/'train_green', 
                    base_path/'train_blue', 
                    base_path/'train_nir',
                    base_path/'train_gt',transform=transformations)


In [ ]:
image,mask=dataset[6000]
plt.imshow(image.permute(1,2,0))

In [ ]:
train_length=int(0.50* len(dataset))

test_length=len(dataset)-train_length


train_dataset,test_dataset=torch.utils.data.random_split(dataset,(train_length,test_length))
val_length=250
test_length=test_length-250
val_set,_=torch.utils.data.random_split(test_dataset,(val_length,test_length))
batch_size= 12

trainloader = DataLoader(train_dataset,
        batch_size=batch_size, shuffle=True,num_workers= 2)
testloader = DataLoader(val_set,
        batch_size=batch_size, shuffle=False,num_workers=2)

In [ ]:

wandb.init(name='Clouds', 
           project='Pretrained_UnNet+WeightedBCE_2',
           notes='RGBNIR', 
           #tags=['Replay-Attack','Cyclic_LR'],
           entity='creganstark')

# WandB Configurations (optional)        
wandb.config.lr = 5e-3 
#model=UNet()
#model=model.double()
#model = model.to(device)

#criterion = FocalTverskyLoss()
criterion=nn.BCEWithLogitsLoss(pos_weight=torch.tensor(2.3053))
optimizer = optim.Adam(model.parameters(),lr  =  wandb.config.lr)# ,
                      #momentum     = 0.9,
                      #nesterov     = True,
                      #weight_decay = 5e-4)

#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)
#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)
  

# Log the network weight histograms (optional)
#wandb.watch(model)

num_epochs = 5
start_time = time.time()
prev_acc=0
for epoch in range(1, num_epochs+1):
    
    loss_train, acc_train,metric_train = train(model, criterion, optimizer, trainloader)
    loss_valid, acc_valid,metric_val = validate(model, criterion, testloader)
    
    print('Epoch: {}  Train Loss: {:.4f}  Train IoU: {:.4f}  Valid Loss: {:.4f}  Valid IoU: {:.4f}'.
          format(epoch, loss_train, acc_train, loss_valid, acc_valid))

    # Log the loss and accuracy values at the end of each epoch
    wandb.log({
        "Epoch": epoch,
        "Train IoU": metric_train['IoU'],
        "Train Dice": metric_train['Dice'],
        "Train Pixel Acc": metric_train['Pixel_Acc'],
        "Train Precision":metric_train["Precision"],
        "Train Recall": metric_train['Recall'],
        "Train Specificity":metric_train['Specificity'],
        "Train Loss": loss_train,
        "Val IoU": metric_val['IoU'],
        "Val Dice ": metric_val['Dice'],
        "Val Pixel Acc": metric_val['Pixel_Acc'],
        "Val Precision":metric_val["Precision"],
        "Val Recall": metric_val['Recall'],
        "Val Specificity":metric_val['Specificity'],
        "Val Loss" : loss_valid
       })
    if acc_valid>prev_acc:
      prev_acc=acc_valid
      paths = "model"+str(acc_valid)+".pt"
      print('Saving Model')
      torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'val_loss': loss_valid,
          'val_acc':acc_valid,
          'train_acc':acc_train,
          'loss_acc':loss_train,
          }, str(paths))

print("Time Elapsed : {:.4f}s".format(time.time() - start_time))

In [ ]:
criterion=nn.BCEWithLogitsLoss(pos_weight=torch.tensor(2.3053))
optimizer = optim.Adam(model.parameters(),lr  =  1e-2)# ,
num_epochs = 5
start_time = time.time()
prev_acc=0
for epoch in range(1, num_epochs+1):
    
    loss_train, acc_train,metric_train = train(model, criterion, optimizer, trainloader)
    loss_valid, acc_valid,metric_val = validate(model, criterion, testloader)
    

In [ ]:
def Metrics(inputs, targets):
    
    inputs = torch.sigmoid(inputs)
    inputs=inputs.round().int()
    targets=targets.int()
    smooth = 1.0
    inputs = inputs.view(-1)
    targets = targets.view(-1)
    intersection = (inputs & targets).float().sum()
    TP=intersection
    FP = ((1-targets) & inputs).float().sum()
    FN = (targets & (1-inputs)).float().sum()
    TN = ((1-targets) & (1-inputs)).float().sum()
    total = (inputs + targets).float().sum()
    union = total - intersection 
    IoU = ((intersection + smooth)/(union + smooth))
    # 'Dice':dice, 'Pixel_Acc': acc, 'Precision': precision,'Recall': recall, 'Specificity':specificity}
    #return metrics
    
    dice = (2.0 * intersection + smooth) / (total + smooth)
    IoU = ((intersection + smooth)/(union + smooth))
    valid = (targets >= 0)
    acc_sum = (valid * (inputs == targets)).sum()
    valid_sum = valid.sum()
    #acc2=(inputs.argmax(dim=1) == targets.float().mean()
    acc = (float(acc_sum) / (valid_sum + 1e-10))
    precision = (intersection/(FP+intersection+1e-5))
    recall = (intersection/(FN+intersection+1e-5))
    specificity = (TN/(TN+FP+1e-5))
    metrics={'IoU':IoU,'Dice':dice, 'Pixel_Acc': acc, 'Precision': precision,'Recall': recall, 'Specificity':specificity}
    return metrics
def validate2(network,criterion): 
    train_dataset,test_dataset=torch.utils.data.random_split(dataset,(6000,2400))
    

    batch_size= 12

    
    testloader = DataLoader(test_dataset,
        batch_size=batch_size, shuffle=False,num_workers=2)
    
    
    loss_valid = 0
    acc_valid = 0    
    acc_pix = 0
    acc_dice=0
    precision=0
    recall=0
    specificity=0
    network.eval()  

    for step in tqdm(range(len(testloader))):

        images , masks = next(iter(testloader))
        
        # move the images and labels to GPU
        images = images.to(device)
        masks = masks.to(device)
        
        
        pred = network(images)
        
        
        loss_valid_step = criterion(pred , masks)
       
        val_metrics=Metrics(pred,masks)
        
        #acc_val = val_metrics
        acc_valid_step=val_metrics['IoU']
        acc_valid_dice=val_metrics['Dice']
        acc_valid_pixel=val_metrics['Pixel_Acc']
        acc_precision=val_metrics['Precision']
        acc_recall = val_metrics['Recall']
        acc_speicificity = val_metrics['Specificity']
        precision+=acc_precision
        recall+=acc_recall
        specificity+=acc_speicificity
        loss_valid += loss_valid_step.item()
        acc_valid += acc_valid_step
        acc_pix+=acc_valid_pixel
        acc_dice+=acc_valid_dice
        
        
        #print(loss_tvalid_step,masks.shape)
        #print(acc_val)

    loss_valid /= len(testloader)
    acc_valid /= len(testloader)
    acc_pix/= len(testloader)
    acc_dice/= len(testloader) 
    precision/=len(testloader) 
    recall/=len(testloader) 
    specificty/=len(testloader) 
    

    return loss_valid,acc_valid,acc_pix,acc_dice,precision,recall,specificity

In [ ]:
loss_valid,acc_valid,acc_pix,acc_dice,precision,recall,specificity=validate2(model,criterion)

In [ ]:
paths='finalbce50.pt'
torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'val_loss': loss_valid,
          'val_acc':acc_valid,
          'train_acc':acc_train,
          'loss_acc':loss_train,
          }, str(paths))

In [ ]:
import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLink
FileLink(r'finalbce50.pt')

In [ ]:
#optimizer = 'optimizer_state_dict': optimizer.state_dict()
filename='../input/checkpointsures50/25epochsunet50.pt'
checkpoint = torch.load(filename)

In [ ]:
optimizer = optim.Adam(model.parameters(),lr  = 5e-3)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
optimizer

In [ ]:

#wandb.init(name='Clouds', 
           #project='Pretrained_UnNet+WeightedBCE_2',
           #notes='RGBNIR', 
           #tags=['Replay-Attack','Cyclic_LR'],
           #entity='creganstark')

# WandB Configurations (optional)        
#wandb.config.lr = 7e-3 
#model=UNet()
model=model.double()
model = model.to(device)

#criterion = FocalTverskyLoss()
criterion=nn.BCEWithLogitsLoss(pos_weight=torch.tensor(2.3053))
optimizer = optim.Adam(model.parameters(),lr  = 0.007 )# ,
                      #momentum     = 0.9,
                      #nesterov     = True,
                      #weight_decay = 5e-4)

#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)
#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)
  

# Log the network weight histograms (optional)
#wandb.watch(model)

num_epochs = 5
start_time = time.time()
prev_acc=0
for epoch in range(1, num_epochs+1):
    
    loss_train, acc_train,metric_train = train(model, criterion, optimizer, trainloader)
    loss_valid, acc_valid,metric_val = validate(model, criterion, testloader)
    
    print('Epoch: {}  Train Loss: {:.4f}  Train IoU: {:.4f}  Valid Loss: {:.4f}  Valid IoU: {:.4f}'.
          format(epoch, loss_train, acc_train, loss_valid, acc_valid))

    # Log the loss and accuracy values at the end of each epoch
    wandb.log({
        "Epoch": epoch,
        "Train IoU": metric_train['IoU'],
        "Train Dice": metric_train['Dice'],
        "Train Pixel Acc": metric_train['Pixel_Acc'],
        "Train Precision":metric_train["Precision"],
        "Train Recall": metric_train['Recall'],
        "Train Specificity":metric_train['Specificity'],
        "Train Loss": loss_train,
        "Val IoU": metric_val['IoU'],
        "Val Dice ": metric_val['Dice'],
        "Val Pixel Acc": metric_val['Pixel_Acc'],
        "Val Precision":metric_val["Precision"],
        "Val Recall": metric_val['Recall'],
        "Val Specificity":metric_val['Specificity'],
        "Val Loss" : loss_valid
       })
    if acc_valid>prev_acc:
      prev_acc=acc_valid
      paths = "model"+str(acc_valid)+".pt"
      print('Saving Model')
      torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'val_loss': loss_valid,
          'val_acc':acc_valid,
          'train_acc':acc_train,
          'loss_acc':loss_train,
          }, str(paths))

print("Time Elapsed : {:.4f}s".format(time.time() - start_time))

In [ ]:
import os
os.chdir(r'/kaggle/working')

In [ ]:
import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLink
FileLink(r'iou_0.6147.pt')

In [ ]:
def validate2(network,criterion, testloader): 
    loss_valid = 0
    acc_valid = 0    
    acc_pix = 0
    acc_dice=0
    network.eval()  

    for step in tqdm(range(len(testloader))):

        images , masks = next(iter(testloader))
        
        # move the images and labels to GPU
        images = images.to(device)
        masks = masks.to(device)
        
        
        pred = network(images)
        
        
        loss_valid_step = criterion(pred , masks)
       
        val_metrics=Metrics(pred,masks)
        
        #acc_val = val_metrics
        acc_valid_step=val_metrics['IoU']
        acc_valid_dice=val_metrics['Dice']
        acc_valid_pixel=val_metrics['Pixel_Acc']
        loss_valid += loss_valid_step.item()
        acc_valid += acc_valid_step
        acc_pix+=acc_valid_pixel
        acc_dice+=acc_valid_dice
        
        
        #print(loss_tvalid_step,masks.shape)
        #print(acc_val)

    loss_valid /= len(testloader)
    acc_valid /= len(testloader)
    acc_pix/= len(testloader)
    acc_dice/= len(testloader)   

    

    return loss_valid,acc_valid,acc_pix,acc_dice

In [ ]:
train_length=int(0.70* len(dataset))

test_length=len(dataset)-train_length


train_dataset,test_dataset=torch.utils.data.random_split(dataset,(train_length,test_length))
#val_length=300
#test_length-=300
#val_set,_=torch.utils.data.random_split(test_dataset,(val_length,test_length))
batch_size= 12

trainloader = DataLoader(train_dataset,
        batch_size=batch_size, shuffle=True,num_workers= 2)
testloader = DataLoader(test_dataset,
        batch_size=batch_size, shuffle=False,num_workers=2)

In [ ]:
train_length=int(0.90* len(dataset))

test_length=len(dataset)-train_length


train_dataset,test_dataset=torch.utils.data.random_split(dataset,(train_length,test_length))
val_length=200
test_length-=200
val_set,_=torch.utils.data.random_split(test_dataset,(val_length,test_length))
batch_size= 12

trainloader = DataLoader(train_dataset,
        batch_size=batch_size, shuffle=True,num_workers= 2)
testloader = DataLoader(val_set,
        batch_size=batch_size, shuffle=False,num_workers=2)

In [ ]:
%cd ./

In [ ]:
#filename = 'modeltensor(0.8413, device='cuda:0').pt
filename = 'modeltensor(0.7946, device='+'-cuda'+':'+'0-'+').pt'
filename=filename.replace('-',"'")
filename='../input/checkpointsures50/25epochsunet50.pt'
checkpoint = torch.load(filepath)
def load_checkpoint(model,filepath):
    checkpoint = torch.load(filepath)
    model.load_state_dict(checkpoint['model_state_dict'])
    #for parameter in model.parameters():
        #parameter.requires_grad = True
    return model
model=load_checkpoint(model,filename)

In [ ]:
loss_valid, acc_valid,acc_pix,acc_dice = validate(model,criterion, testloader)

In [ ]:

wandb.init(name='Clouds', 
           project='Pretrained_UnNet+WeightedBCE_2',
           notes='RGBNIR', 
           #tags=['Replay-Attack','Cyclic_LR'],
           entity='creganstark')

# WandB Configurations (optional)        
wandb.config.lr = 5e-3 
#model=UNet()
#model=model.double()
#model = model.to(device)

#criterion = FocalTverskyLoss()
criterion=nn.BCEWithLogitsLoss(pos_weight=torch.tensor(2.3053))
optimizer = optim.Adam(model.parameters(),lr  =  wandb.config.lr)# ,
                      #momentum     = 0.9,
                      #nesterov     = True,
                      #weight_decay = 5e-4)

#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)
#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)
  

# Log the network weight histograms (optional)
#wandb.watch(model)

num_epochs = 5
start_time = time.time()
prev_acc=0
for epoch in range(1, num_epochs+1):
    
    loss_train, acc_train,metric_train = train(model, criterion, optimizer, trainloader)
    loss_valid, acc_valid,metric_val = validate(model, criterion, testloader)
    
    print('Epoch: {}  Train Loss: {:.4f}  Train IoU: {:.4f}  Valid Loss: {:.4f}  Valid IoU: {:.4f}'.
          format(epoch, loss_train, acc_train, loss_valid, acc_valid))

    # Log the loss and accuracy values at the end of each epoch
    wandb.log({
        "Epoch": epoch,
        "Train IoU": metric_train['IoU'],
        "Train Dice": metric_train['Dice'],
        "Train Pixel Acc": metric_train['Pixel_Acc'],
        "Train Precision":metric_train["Precision"],
        "Train Recall": metric_train['Recall'],
        "Train Specificity":metric_train['Specificity'],
        "Train Loss": loss_train,
        "Val IoU": metric_val['IoU'],
        "Val Dice ": metric_val['Dice'],
        "Val Pixel Acc": metric_val['Pixel_Acc'],
        "Val Precision":metric_val["Precision"],
        "Val Recall": metric_val['Recall'],
        "Val Specificity":metric_val['Specificity'],
        "Val Loss" : loss_valid
       })
    if acc_valid>prev_acc:
      prev_acc=acc_valid
      paths = "model"+str(acc_valid)+".pt"
      print('Saving Model')
      torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'val_loss': loss_valid,
          'val_acc':acc_valid,
          'train_acc':acc_train,
          'loss_acc':loss_train,
          }, str(paths))

print("Time Elapsed : {:.4f}s".format(time.time() - start_time))

In [ ]:
loss_valid, acc_valid,acc_pix,acc_dice = validate(model,criterion, testloader)

In [ ]:
lr = 9e-3 
criterion=nn.BCEWithLogitsLoss(pos_weight=torch.tensor(2.3053))
optimizer = optim.Adam(model.parameters(),lr  =  lr)# ,
num_epochs = 5
start_time = time.time()
prev_acc=0
for epoch in range(1, num_epochs+1):
    
    loss_train, acc_train,metric_train = train(model, criterion, optimizer, trainloader)
    loss_valid, acc_valid,metric_val = validate(model, criterion, testloader)
    
    print('Epoch: {}  Train Loss: {:.4f}  Train IoU: {:.4f}  Valid Loss: {:.4f}  Valid IoU: {:.4f}'.
          format(epoch, loss_train, acc_train, loss_valid, acc_valid))

In [ ]:
len(train_dataset)

In [ ]:
loss_valid, acc_valid,acc_pix,acc_dice

In [ ]:
loss_valid, acc_valid,acc_pix,acc_dice

In [ ]:
loss_valid, acc_valid,acc_pix,acc_dice

In [ ]:
loss_valid, acc_valid,acc_pix,acc_dice

In [ ]:
loss_valid, acc_valid,acc_pix,acc_dice

In [ ]:
loss_valid, acc_valid,metric_val

In [ ]:
#filename = 'modeltensor(0.8413, device='cuda:0').pt
filename = 'modeltensor(0.7946, device='+'-cuda'+':'+'0-'+').pt'
filename=filename.replace('-',"'")
#filename='epoch20checkpoint.pth'
def load_checkpoint(model,filepath):
    checkpoint = torch.load(filepath)
    model.load_state_dict(checkpoint['model_state_dict'])
    #for parameter in model.parameters():
        #parameter.requires_grad = True
    return model
model=load_checkpoint(model,filename)

In [ ]:
paths='iou_0.6.pt'
torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'val_loss': loss_valid,
          'val_acc':acc_valid,
          'train_acc':acc_train,
          'loss_acc':loss_train,
          }, str(paths))

In [ ]:
loss_valid, acc_valid,metric_val = validate(model,criterion, testloader)

In [ ]:
loss_valid, acc_valid,metric_val = validate(model,criterion, testloader)

In [ ]:
loss_valid, acc_valid,metric_val

In [ ]:
loss_valid, acc_valid,metric_val

In [ ]:
loss_valid, acc_valid,metric_val = validate(model,criterion, testloader)

In [ ]:
loss_valid, acc_valid,metric_val

In [ ]:
loss_valid, acc_valid,metric_val = validate(model,criterion, testloader)

In [ ]:
loss_valid, acc_valid,metric_val

In [ ]:
loss_valid, acc_valid,metric_val = validate(model,criterion, testloader)

In [ ]:
loss_valid, acc_valid,metric_val

In [ ]:

wandb.init(name='Clouds', 
           project='Pretrained_UnNet+WeightedBCE_2',
           notes='RGBNIR', 
           #tags=['Replay-Attack','Cyclic_LR'],
           entity='creganstark')

# WandB Configurations (optional)        
wandb.config.lr = 5e-3 
#model=UNet()
#model=model.double()
#model = model.to(device)

#criterion = FocalTverskyLoss()
criterion=nn.BCEWithLogitsLoss(pos_weight=torch.tensor(2.3053))
optimizer = optim.Adam(model.parameters(),lr  =  wandb.config.lr)# ,
                      #momentum     = 0.9,
                      #nesterov     = True,
                      #weight_decay = 5e-4)

#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)
#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)
  

# Log the network weight histograms (optional)
#wandb.watch(model)

num_epochs = 5
start_time = time.time()
prev_acc=0
for epoch in range(1, num_epochs+1):
    
    loss_train, acc_train,metric_train = train(model, criterion, optimizer, trainloader)
    loss_valid, acc_valid,metric_val = validate(model, criterion, testloader)
    
    print('Epoch: {}  Train Loss: {:.4f}  Train IoU: {:.4f}  Valid Loss: {:.4f}  Valid IoU: {:.4f}'.
          format(epoch, loss_train, acc_train, loss_valid, acc_valid))

    # Log the loss and accuracy values at the end of each epoch
    wandb.log({
        "Epoch": epoch,
        "Train IoU": metric_train['IoU'],
        "Train Dice": metric_train['Dice'],
        "Train Pixel Acc": metric_train['Pixel_Acc'],
        "Train Precision":metric_train["Precision"],
        "Train Recall": metric_train['Recall'],
        "Train Specificity":metric_train['Specificity'],
        "Train Loss": loss_train,
        "Val IoU": metric_val['IoU'],
        "Val Dice ": metric_val['Dice'],
        "Val Pixel Acc": metric_val['Pixel_Acc'],
        "Val Precision":metric_val["Precision"],
        "Val Recall": metric_val['Recall'],
        "Val Specificity":metric_val['Specificity'],
        "Val Loss" : loss_valid
       })
    if acc_valid>prev_acc:
      prev_acc=acc_valid
      paths = "model"+str(acc_valid)+".pt"
      print('Saving Model')
      torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'val_loss': loss_valid,
          'val_acc':acc_valid,
          'train_acc':acc_train,
          'loss_acc':loss_train,
          }, str(paths))

print("Time Elapsed : {:.4f}s".format(time.time() - start_time))

In [ ]:
paths='25epochsunet50'
torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'val_loss': loss_valid,
          'val_acc':acc_valid,
          'train_acc':acc_train,
          'loss_acc':loss_train,
          }, str(paths))

In [ ]:
filename = 'modeltensor(0.8413, device='+'-cuda'+':'+'0-'+').pt'

In [ ]:
filename = 'modeltensor(0.8413, device='+'-cuda'+':'+'0-'+').pt'
filename=filename.replace('-',"'")

In [ ]:
./modeltensor(0.8413, device='cuda:0').pt

In [ ]:
#filename = 'modeltensor(0.8413, device='cuda:0').pt
#filename = 'modeltensor(0.8413, device='+'-cuda'+':'+'0-'+').pt'
#filename=filename.replace('-',"'")
filename='25epochsunet50'
def load_checkpoint(model,filepath):
    checkpoint = torch.load(filepath)
    model.load_state_dict(checkpoint['model_state_dict'])
    #for parameter in model.parameters():
        #parameter.requires_grad = True
    return model
model=load_checkpoint(model,filename)

In [ ]:
train_length=int(0.70* len(dataset))

test_length=len(dataset)-train_length


train_dataset,test_dataset=torch.utils.data.random_split(dataset,(train_length,test_length))
val_length=300
test_length-=300
val_set,_=torch.utils.data.random_split(test_dataset,(val_length,test_length))
batch_size= 12

trainloader = DataLoader(train_dataset,
        batch_size=batch_size, shuffle=True,num_workers= 2)
testloader = DataLoader(val_set,
        batch_size=batch_size, shuffle=False,num_workers=2)

In [ ]:

wandb.init(name='Clouds', 
           project='Pretrained_UnNet+WeightedBCE_2',
           notes='RGBNIR', 
           #tags=['Replay-Attack','Cyclic_LR'],
           entity='creganstark')

# WandB Configurations (optional)        
wandb.config.lr = 5e-3 
#model=UNet()
#model=model.double()
#model = model.to(device)

#criterion = FocalTverskyLoss()
criterion=nn.BCEWithLogitsLoss(pos_weight=torch.tensor(2.3053))
optimizer = optim.Adam(model.parameters(),lr  =  wandb.config.lr)# ,
                      #momentum     = 0.9,
                      #nesterov     = True,
                      #weight_decay = 5e-4)

#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)
#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)
  

# Log the network weight histograms (optional)
#wandb.watch(model)

num_epochs = 5
start_time = time.time()
prev_acc=0
for epoch in range(1, num_epochs+1):
    
    loss_train, acc_train,metric_train = train(model, criterion, optimizer, trainloader)
    loss_valid, acc_valid,metric_val = validate(model, criterion, testloader)
    
    print('Epoch: {}  Train Loss: {:.4f}  Train IoU: {:.4f}  Valid Loss: {:.4f}  Valid IoU: {:.4f}'.
          format(epoch, loss_train, acc_train, loss_valid, acc_valid))

    # Log the loss and accuracy values at the end of each epoch
    wandb.log({
        "Epoch": epoch,
        "Train IoU": metric_train['IoU'],
        "Train Dice": metric_train['Dice'],
        "Train Pixel Acc": metric_train['Pixel_Acc'],
        "Train Precision":metric_train["Precision"],
        "Train Recall": metric_train['Recall'],
        "Train Specificity":metric_train['Specificity'],
        "Train Loss": loss_train,
        "Val IoU": metric_val['IoU'],
        "Val Dice ": metric_val['Dice'],
        "Val Pixel Acc": metric_val['Pixel_Acc'],
        "Val Precision":metric_val["Precision"],
        "Val Recall": metric_val['Recall'],
        "Val Specificity":metric_val['Specificity'],
        "Val Loss" : loss_valid
       })
    if acc_valid>prev_acc:
      prev_acc=acc_valid
      paths = "model"+str(acc_valid)+".pt"
      print('Saving Model')
      torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'val_loss': loss_valid,
          'val_acc':acc_valid,
          'train_acc':acc_train,
          'loss_acc':loss_train,
          }, str(paths))

print("Time Elapsed : {:.4f}s".format(time.time() - start_time))

In [ ]:

wandb.init(name='Clouds', 
           project='Pretrained_UnNet+WeightedBCE_2',
           notes='RGBNIR', 
           #tags=['Replay-Attack','Cyclic_LR'],
           entity='creganstark')

# WandB Configurations (optional)        
wandb.config.lr = 5e-3 
#model=UNet()
#model=model.double()
#model = model.to(device)

#criterion = FocalTverskyLoss()
criterion=nn.BCEWithLogitsLoss(pos_weight=torch.tensor(2.3053))
optimizer = optim.Adam(model.parameters(),lr  =  wandb.config.lr)# ,
                      #momentum     = 0.9,
                      #nesterov     = True,
                      #weight_decay = 5e-4)

#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)
#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)
  

# Log the network weight histograms (optional)
#wandb.watch(model)

num_epochs = 5
start_time = time.time()
prev_acc=0
for epoch in range(1, num_epochs+1):
    
    loss_train, acc_train,metric_train = train(model, criterion, optimizer, trainloader)
    loss_valid, acc_valid,metric_val = validate(model, criterion, testloader)
    
    print('Epoch: {}  Train Loss: {:.4f}  Train IoU: {:.4f}  Valid Loss: {:.4f}  Valid IoU: {:.4f}'.
          format(epoch, loss_train, acc_train, loss_valid, acc_valid))

    # Log the loss and accuracy values at the end of each epoch
    wandb.log({
        "Epoch": epoch,
        "Train IoU": metric_train['IoU'],
        "Train Dice": metric_train['Dice'],
        "Train Pixel Acc": metric_train['Pixel_Acc'],
        "Train Precision":metric_train["Precision"],
        "Train Recall": metric_train['Recall'],
        "Train Specificity":metric_train['Specificity'],
        "Train Loss": loss_train,
        "Val IoU": metric_val['IoU'],
        "Val Dice ": metric_val['Dice'],
        "Val Pixel Acc": metric_val['Pixel_Acc'],
        "Val Precision":metric_val["Precision"],
        "Val Recall": metric_val['Recall'],
        "Val Specificity":metric_val['Specificity'],
        "Val Loss" : loss_valid
       })
    if acc_valid>prev_acc:
      prev_acc=acc_valid
      paths = "model"+str(acc_valid)+".pt"
      print('Saving Model')
      torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'val_loss': loss_valid,
          'val_acc':acc_valid,
          'train_acc':acc_train,
          'loss_acc':loss_train,
          }, str(paths))

print("Time Elapsed : {:.4f}s".format(time.time() - start_time))

In [ ]:

wandb.init(name='Clouds', 
           project='Pretrained_UnNet+WeightedBCE',
           notes='RGBNIR', 
           #tags=['Replay-Attack','Cyclic_LR'],
           entity='creganstark')

# WandB Configurations (optional)        
wandb.config.lr = 7e-3 
#model=UNet()
#model=model.double()
#model = model.to(device)

#criterion = FocalTverskyLoss()
criterion=nn.BCEWithLogitsLoss(pos_weight=torch.tensor(2.3053))
optimizer = optim.Adam(model.parameters(),lr  =  wandb.config.lr)# ,
                      #momentum     = 0.9,
                      #nesterov     = True,
                      #weight_decay = 5e-4)

#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)
#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)
  

# Log the network weight histograms (optional)
#wandb.watch(model)

num_epochs = 20
start_time = time.time()
prev_acc=0
for epoch in range(1, num_epochs+1):
    
    loss_train, acc_train,metric_train = train(model, criterion, optimizer, trainloader)
    loss_valid, acc_valid,metric_val = validate(model, criterion, testloader)
    
    print('Epoch: {}  Train Loss: {:.4f}  Train IoU: {:.4f}  Valid Loss: {:.4f}  Valid IoU: {:.4f}'.
          format(epoch, loss_train, acc_train, loss_valid, acc_valid))

    # Log the loss and accuracy values at the end of each epoch
    wandb.log({
        "Epoch": epoch,
        "Train IoU": metric_train['IoU'],
        "Train Dice": metric_train['Dice'],
        "Train Pixel Acc": metric_train['Pixel_Acc'],
        "Train Precision":metric_train["Precision"],
        "Train Recall": metric_train['Recall'],
        "Train Specificity":metric_train['Specificity'],
        "Train Loss": loss_train,
        "Val IoU": metric_val['IoU'],
        "Val Dice ": metric_val['Dice'],
        "Val Pixel Acc": metric_val['Pixel_Acc'],
        "Val Precision":metric_val["Precision"],
        "Val Recall": metric_val['Recall'],
        "Val Specificity":metric_val['Specificity'],
        "Val Loss" : loss_valid
       })
    if acc_valid>prev_acc:
      prev_acc=acc_valid
      paths = "model"+str(acc_valid)+".pt"
      print('Saving Model')
      torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'val_loss': loss_valid,
          'val_acc':acc_valid,
          'train_acc':acc_train,
          'loss_acc':loss_train,
          }, str(paths))

print("Time Elapsed : {:.4f}s".format(time.time() - start_time))

In [ ]:

wandb.init(name='Clouds', 
           project='Pretrained_UnNet+WeightedBCE',
           notes='RGBNIR', 
           #tags=['Replay-Attack','Cyclic_LR'],
           entity='creganstark')

# WandB Configurations (optional)        
wandb.config.lr = 7e-3 
#model=UNet()
#model=model.double()
#model = model.to(device)

#criterion = FocalTverskyLoss()
criterion=nn.BCEWithLogitsLoss(pos_weight=torch.tensor(2.3053))
optimizer = optim.Adam(model.parameters(),lr  =  wandb.config.lr)# ,
                      #momentum     = 0.9,
                      #nesterov     = True,
                      #weight_decay = 5e-4)

#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)
#scheduler = optim.lr_scheduler.CyclicLR(optimizer,base_lr=wandb.config.lr,max_lr=1e-3 ,step_size_up=2000)
  

# Log the network weight histograms (optional)
#wandb.watch(model)

num_epochs = 20
start_time = time.time()
prev_acc=0
for epoch in range(1, num_epochs+1):
    
    loss_train, acc_train,metric_train = train(model, criterion, optimizer, trainloader)
    loss_valid, acc_valid,metric_val = validate(model, criterion, testloader)
    
    print('Epoch: {}  Train Loss: {:.4f}  Train IoU: {:.4f}  Valid Loss: {:.4f}  Valid IoU: {:.4f}'.
          format(epoch, loss_train, acc_train, loss_valid, acc_valid))

    # Log the loss and accuracy values at the end of each epoch
    wandb.log({
        "Epoch": epoch,
        "Train IoU": metric_train['IoU'],
        "Train Dice": metric_train['Dice'],
        "Train Pixel Acc": metric_train['Pixel_Acc'],
        "Train Precision":metric_train["Precision"],
        "Train Recall": metric_train['Recall'],
        "Train Specificity":metric_train['Specificity'],
        "Train Loss": loss_train,
        "Val IoU": metric_val['IoU'],
        "Val Dice ": metric_val['Dice'],
        "Val Pixel Acc": metric_val['Pixel_Acc'],
        "Val Precision":metric_val["Precision"],
        "Val Recall": metric_val['Recall'],
        "Val Specificity":metric_val['Specificity'],
        "Val Loss" : loss_valid
       })
    if acc_valid>prev_acc:
      prev_acc=acc_valid
      paths = "model"+str(acc_valid)+".pt"
      print('Saving Model')
      torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'val_loss': loss_valid,
          'val_acc':acc_valid,
          'train_acc':acc_train,
          'loss_acc':loss_train,
          }, str(paths))

print("Time Elapsed : {:.4f}s".format(time.time() - start_time))

In [ ]:
images,labels=(next(iter(trainloader)))
images = images.to(device)
labels = masks.to(device)
pred = network(images)

In [ ]:
!nvidia-smi